<div class="alert alert-info" >
    
# <div class="alert alert-success" > Combining and Reshaping Data
    
Everything that we did focused upon working within the data of a single DataFrame or Series object, and keeping the same structure of data within those objects. Once the data is tidied up, it will be likely that we will then need to use this data either to combine multiple sets of data, or to reorganize the structure of the data by moving data in and out of indexes.

We'll cover two general categories of topics: 
    
- The first two sections will cover the capabilities provided by pandas to combine the data from multiple pandas objects together. Combination of data in pandas is performed by <b>concatenating</b> two sets of data, where data is combined simply along either axes but without regard to relationships in the data. Or data can be combined using relationships in the data by using a pandas capability referred to as <b>merging</b>, which provides join operations that are similar to those in many
relational databases.
    
- The remaining sections will examine the three primary means reshaping data in pandas. These will examine the processes of pivoting, stacking and unstacking, and melting of data.

    <b>Pivoting</b> allows us to restructure pandas data similarly to how spreadsheets pivot data by creating new index levels and moving data into columns based upon values (or vice-versa).
    
    <b>Stacking and unstacking</b> are similar to pivoting, but allow us to pivot data organized with multiple levels of indexes. 
    
    <b>Melting</b> allows us to restructure data into unique ID-variable-measurement combinations that are or required for many statistical analyses.    

### <div class= "alert alert-warning"> Setting up the IPython notebook
    
To utilize the examples in this chapter we will need to include the following imports
and settings.



In [1]:
import numpy as np
import pandas as pd
import datetime
    
# Set some pandas options for controlling output
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)

<div class="alert alert-success" >
    
## <div class= "alert alert-info">Concatenating data
    
Concatenation in pandas is the process of either adding rows to the end of an existing Series or DataFrame object or adding additional columns to a DataFrame. 
    
In pandas, concatenation is performed via the pandas function <span style="color:red">pd.concat()</span>. The function will perform the operation on a specific axis and as we will see, will also perform any required set logic involved in aligning along that axis.
    
The general syntax to concatenate data is to pass a list of objects to pd.concat(). The following performs a concatenation of two Series objects:

In [10]:
s1 = pd.Series(np.arange(0, 3))
s2 = pd.Series(np.arange(5, 8))
print('s1:\n',s1,'\n')
print('s2:\n',s2,'\n')

s1:
 0    0
1    1
2    2
dtype: int32 

s2:
 0    5
1    6
2    7
dtype: int32 



In [11]:
pd.concat([s1, s2])         # concatenate s1 & s2 together 

0    0
1    1
2    2
0    5
1    6
2    7
dtype: int32

In [26]:
# create two DataFrame objects to concatenate. using the same index labels and column names, but different values
df1 = pd.DataFrame(np.arange(9).reshape(3, 3),columns=['a', 'b', 'c'])
df2 = pd.DataFrame(np.arange(9, 18).reshape(3, 3),columns=['a', 'b', 'c'])
df1

   a  b  c
0  0  1  2
1  3  4  5
2  6  7  8

In [27]:
df2

    a   b   c
0   9  10  11
1  12  13  14
2  15  16  17

In [23]:
dfc = pd.concat([df1, df2])    #  concat df1 & df2
dfc

    a   b   c
0   0   1   2
1   3   4   5
2   6   7   8
0   9  10  11
1  12  13  14
2  15  16  17

The process of concatenating the two DataFrame objects will first identify the set of columns formed by aligning the labels in the columns, effectively determining the union of the column names. The resulting DataFrame object will then consist of that
set of columns, and columns with identical names will not be duplicated.

Rows will be then be added to the result, in the order of the each of the objects passed to pd.concat(). If a column in the result does not exist in the object being copied, NaN values will be filled in those locations. Duplicate row index labels can occur.

The following demonstrates the alignment of two DataFrame objects during concatenation that both have columns in common (a and c) and also have distinct columns (b in df1, and d in df2):

In [28]:
# demonstrate concatenating two DataFrame objects with different columns
df1 = pd.DataFrame(np.arange(9).reshape(3, 3),columns=['a', 'b', 'c'])
df2 = pd.DataFrame(np.arange(9, 18).reshape(3, 3),columns=['a', 'c', 'd'])
df1

   a  b  c
0  0  1  2
1  3  4  5
2  6  7  8

## 

In [29]:
df2

    a   c   d
0   9  10  11
1  12  13  14
2  15  16  17

In [30]:
# do the concat, NaN values will be filled in for the d column for df1 and b column for df2
pd.concat([df1, df2])

    a    b   c     d
0   0  1.0   2   NaN
1   3  4.0   5   NaN
2   6  7.0   8   NaN
0   9  NaN  10  11.0
1  12  NaN  13  14.0
2  15  NaN  16  17.0

It is possible to give each group of data in the result its own name using the <span style="color:green"><b>keys</b></span> parameter. This creates a hierarchical index on the DataFrame object that lets you refer to each group of data independently via the DataFrame objects' .ix property. This is convenient if you later need to determine where data in the concatenated DataFrame object came from.

The following sample demonstrates this concept by assigning names to each original DataFrame object and then retrieving the rows that originated in the df2 object, which are keyed with the label 'df2'. The following code demonstrates this labeling
and also retrieves just the rows that originated in df2:

In [100]:
C = pd.concat([df1, df2], keys=['df1', 'df2'])
C

        a    b   c     d
df1 0   0  1.0   2   NaN
    1   3  4.0   5   NaN
    2   6  7.0   8   NaN
df2 0   9  NaN  10  11.0
    1  12  NaN  13  14.0
    2  15  NaN  16  17.0

Accessinf df1 from C dataframe:

In [102]:
C.loc['df1']

   a    b  c   d
0  0  1.0  2 NaN
1  3  4.0  5 NaN
2  6  7.0  8 NaN

In [42]:
c = pd.concat([df1, df2], keys=['df1', 'df2'], names=['DF_ID', 'R_ID'])
c

             a    b   c     d
DF_ID R_ID                   
df1   0      0  1.0   2   NaN
      1      3  4.0   5   NaN
      2      6  7.0   8   NaN
df2   0      9  NaN  10  11.0
      1     12  NaN  13  14.0
      2     15  NaN  16  17.0

The pd.concat() function also allows you to specify the axis on which to apply the concatenation. The following concatenates the two DataFrame objects along the columns axis:

In [43]:
pd.concat([df1, df2], axis=1) # concat df1 and df2 along columns aligns on row labels, has duplicate columns

   a  b  c   a   c   d
0  0  1  2   9  10  11
1  3  4  5  12  13  14
2  6  7  8  15  16  17

Note that the result now contains duplicate columns. The concatenation first aligns by the row index labels of each DataFrame object, and then fills in the columns from the first DataFrame object and then the second. The columns are not aligned and
result in duplicate values. The same rules of alignment and filling of NaN values apply in this case, except that they are applied to the rows' index labels.

The following demonstrates a concatenation along the columns axis with two DataFrame objects that have row index labels in
common (2 and 3) along with disjoint rows (0 in df1 and 4 in df3). Additionally, some of the columns in df3 overlap with df1 (a) as well as being disjoint (d):

In [52]:
df1

   a  b  c
0  0  1  2
1  3  4  5
2  6  7  8

# 

In [53]:
df3 = pd.DataFrame(np.arange(20, 26).reshape(3, 2),columns=['a', 'd'],index=[2, 3, 4])
df3

    a   d
2  20  21
3  22  23
4  24  25

In [55]:
pd.concat([df1, df3], axis=1)  

     a    b    c     a     d
0  0.0  1.0  2.0   NaN   NaN
1  3.0  4.0  5.0   NaN   NaN
2  6.0  7.0  8.0  20.0  21.0
3  NaN  NaN  NaN  22.0  23.0
4  NaN  NaN  NaN  24.0  25.0

<mark>A concatenation of two or more DataFrame objects actually performs an outer join operation along the index labels on the axis opposite to the one specified</mark>. This makes the result of the concatenation similar to having performed a union of those index labels, and then data is filled based on the alignment of those labels to the source objects.

The type of join can be changed to an inner join and can be performed by specifying<span style="color:green"><b> join='inner' </b></span> as the parameter. <mark>The inner join then logically performs an intersection instead of a union</mark>. 

- <span style="color:green"><b> join='outer': </b></span> Union of two data frames along the specified axis

- <span style="color:green"><b> join='inner': </b></span> intersection of two data frames along the specified axis

The following demonstrates this and results in a single row because 2 is the only row index label in common:

In [56]:
pd.concat([df1, df3], axis=1, join='inner')   # do an inner join instead of outer results in one row


   a  b  c   a   d
2  6  7  8  20  21

It is also possible to use label groups of data along the columns using the keys parameter when applying the concatenation along axis=1.

In [57]:

df = pd.concat([df1, df2],axis=1,keys=['df1', 'df2'])    # add keys to the columns
df

  df1       df2        
    a  b  c   a   c   d
0   0  1  2   9  10  11
1   3  4  5  12  13  14
2   6  7  8  15  16  17

The different groups can be accessed using the <span style="color:red">.get()</span> method that is used to access individual columns

In [76]:
df.get('df1')

   a  b  c
0  0  1  2
1  3  4  5
2  6  7  8

A DataFrame (and Series) object also contains an <span style="color:red">.append()</span> method, which will concatenate the two specified DataFrame objects along the row index labels.

In [77]:
df1.append(df2) # append does a concatenate along axis=0 duplicate row index labels can result

C:\Users\Ayesha\AppData\Local\Temp\ipykernel_9752\3348495838.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df2) # append does a concatenate along axis=0 duplicate row index labels can result


    a    b   c     d
0   0  1.0   2   NaN
1   3  4.0   5   NaN
2   6  7.0   8   NaN
0   9  NaN  10  11.0
1  12  NaN  13  14.0
2  15  NaN  16  17.0

As with a concatenation on axis=1, the index labels in the rows are copied without consideration of the creation of duplicates, and the columns labels are joined in a manner which ensures no duplicate column name is included in the result. If you would like to ensure that the resulting index does not have duplicates but preserves all of the rows, you can use the   <span style="color:green"><b>ignore_index=True</b></span> parameter. This essentially returns the same result except with new Int64Index:

In [78]:
# remove duplicates in the result index by ignoring the index labels in the source DataFrame objects
df1.append(df2, ignore_index=True)

C:\Users\Ayesha\AppData\Local\Temp\ipykernel_9752\3010342201.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df2, ignore_index=True)


    a    b   c     d
0   0  1.0   2   NaN
1   3  4.0   5   NaN
2   6  7.0   8   NaN
3   9  NaN  10  11.0
4  12  NaN  13  14.0
5  15  NaN  16  17.0

<div class="alert alert-success" >
    
## <div class= "alert alert-info">Merging and joining data
    
pandas allows the merging of pandas objects with database-like join operations using the <span style="color:red">pd.merge()</span> function and the <span style="color:red">.merge()</span> method of a DataFrame object. These joins are high performance and are performed in memory. A merge combines the data of two pandas objects by finding matching values in one or more columns
or row indexes. It then returns a new object that represents a combination of the data from both based on relational-database-like join semantics applied to those values.
    
Merges are useful as they allow us to model a single DataFrame for each type of data (one of the rules of having tidy data) but to be able to relate data in different DataFrame objects using values existing in both sets of data. 
    
## An overview of merges
A practical and probably canonical example would be that of looking up customer names from orders. To demonstrate this in pandas, we will use the following two DataFrame objects, where one represents a list of customer details, and the other
represents the orders made by customers and what day the order was made. They will be related to each other using the CustomerID columns in each:   

In [84]:
# Customers

customers_dic = {'CustomerID': [10, 11],
                 'Name': ['Mike', 'Marcia'],
                 'Address': ['Address for Mike','Address for Marcia']}

customers = pd.DataFrame(customers_dic)
customers

   CustomerID    Name             Address
0          10    Mike    Address for Mike
1          11  Marcia  Address for Marcia

In [85]:
# Orders made by customers. They are related to customers by CustomerID

orders_dic = {'CustomerID': [10, 11, 10],
              'OrderDate': [datetime.date(2014, 12, 1),datetime.date(2014, 12, 1),datetime.date(2014, 12, 1)]}

orders = pd.DataFrame(orders_dic)
orders

   CustomerID   OrderDate
0          10  2014-12-01
1          11  2014-12-01
2          10  2014-12-01

Now suppose we would like to ship the orders to the customers. We would need to merge the orders data with the customers detail data to determine the address for each order. In pandas, this can be easily performed with the following statement:

In [86]:
customers.merge(orders)     # merge customers and orders so we can ship the items

   CustomerID    Name             Address   OrderDate
0          10    Mike    Address for Mike  2014-12-01
1          10    Mike    Address for Mike  2014-12-01
2          11  Marcia  Address for Marcia  2014-12-01

pandas has done something magical for us here by being able to accomplish this with such a simple piece of code. What pandas has done is realized that our customers and orders objects both have a column named CustomerID. With this knowledge, it uses common values found in that column of both DataFrame objects to relate the data in both and form the merged data based on inner join semantics. To be even more detailed, what pandas has specifically done is the following:

1. Determines the columns in both customers and orders with common labels. These columns are treated as the keys to perform the join.

2. It creates a new DataFrame whose columns are the labels from the keys identified in step 1, followed by all of the non-key labels from both objects. 

3. It matches values in the key columns of both DataFrame objects.

4. It then creates a row in the result for each set of matching labels.

5. It then copies the data from those matching rows from each source object into that respective row and columns of the result.

6. It assigns a new Int64Index to the result.

The join in a merge can use values from multiple columns. To demonstrate, the following creates two DataFrame objects and performs the merge where pandas decides to use the values in the key1 and key2 columns of both objects:

In [87]:
# data to be used in the remainder of this section's examples

left_data = {'key1': ['a', 'b', 'c'],
             'key2': ['x', 'y', 'z'],
             'lval1': [ 0, 1, 2]}

right_data = {'key1': ['a', 'b', 'c'],
              'key2': ['x', 'a', 'z'],
              'rval1': [ 6, 7, 8 ]}

left = pd.DataFrame(left_data, index=[0, 1, 2])
right = pd.DataFrame(right_data, index=[1, 2, 3])
left

  key1 key2  lval1
0    a    x      0
1    b    y      1
2    c    z      2

In [88]:
right

  key1 key2  rval1
1    a    x      6
2    b    a      7
3    c    z      8

In [89]:
left.merge(right)     # demonstrate merge without specifying columns to merg this will implicitly merge on all common columns


  key1 key2  lval1  rval1
0    a    x      0      6
1    c    z      2      8

This merge identifies key1 and key2 columns as common in both DataFrame objects and hence uses them for the merge. The matching tuples of values in both DataFrame objects for these columns are (a, x) and (c, z) and therefore this results
in two rows of values.

To explicitly specify which column use to relate the objects, use the on parameter. The following performs a merge using only the values in the key1 column of both DataFrame objects:


In [90]:
left.merge(right, on='key1')# demonstrate merge using an explicit column on needs the value to be in both DataFrame objects

  key1 key2_x  lval1 key2_y  rval1
0    a      x      0      x      6
1    b      y      1      a      7
2    c      z      2      z      8

Comparing this result to the previous example, as only the values in the key1 column were used to relate the data in the two objects, the result now has three rows as there are matching a, b, and c values in that single column of both objects.

The on parameter can also be given a list of column names. The following reverts to using both the key1 and key2 columns, resulting in being identical the earlier example where those two columns where implicitly identified by pandas:

In [91]:
left.merge(right, on=['key1', 'key2'])# merge explicitly using two columns

  key1 key2  lval1  rval1
0    a    x      0      6
1    c    z      2      8

The columns specified with on need to exist in both DataFrame objects. If you would like to merge based on columns with different names in each object, you can use the left_on and right_on parameters, passing the name or names of columns to each
respective parameter.

To perform a merge with the labels of the row indexes of the two DataFrame objects, use
the left_index=True and right_index=True parameters (both need to be specified):

In [92]:
pd.merge(left, right, left_index=True, right_index=True) # join on the row indices of both matrices

  key1_x key2_x  lval1 key1_y key2_y  rval1
1      b      y      1      a      x      6
2      c      z      2      b      a      7

This has identified that the index labels in common are 1 and 2, so the resulting DataFrame has two rows with these values and labels in the index. pandas then creates a column in the result for every column in both objects and then copies the values.

As both DataFrame objects had a column with an identical name, key, the columns in the result have the _x and _y suffixes appended to them to identify the DataFrame they originated from. _x is for left and _y for right. You can specify these suffixes using the suffixes parameter and passing a two-item sequence.

### <div class= "alert alert-warning">Specifying the join semantics of a merge operation
    
<mark>The default type of join performed by pd.merge() is an inner join</mark>. To use another join method, the method of join to be used can be specified using the below parameter of the pd.merge() function (or the .merge() method). The valid options are:
    
• <b>inner:</b> This is the intersection of keys from both DataFrame objects
    
• <b>outer:</b> This is the union of keys from both DataFrame objects
    
• <b>left: </b>This only uses keys from the left DataFrame 
    
• <b>right:</b> This only uses keys from the right DataFrame
    
As we have seen, an inner join is the default and will return a merge of the data from both DataFrame objects only where the values match. 
    
An outer join contrasts, in that it will return both the merge of the matched rows and the unmatched values from both the left and right DataFrame objects, but with NaN 
    filled in the unmatched portion. The following code demonstrates an outer join:

In [93]:
left.merge(right, how='outer') # outer join, merges all matched data, and fills unmatched items with NaN

  key1 key2  lval1  rval1
0    a    x    0.0    6.0
1    b    y    1.0    NaN
2    c    z    2.0    8.0
3    b    a    NaN    7.0

A left join will return the merge of the rows that satisfy the join of the values in the specified columns, and also returns the unmatched rows from only left:

In [95]:
# left join, merges all matched data, and only fills unmatched
# items from the left dataframe with NaN filled for the
# unmatched items in the result
# rows with labels 0 and 2
# match on key1 and key2 the row with label 1 is from left
left.merge(right, how='left')

  key1 key2  lval1  rval1
0    a    x      0    6.0
1    b    y      1    NaN
2    c    z      2    8.0

A right join will return the merge of the rows that satisfy the join of the values in the specified columns, and also returns the unmatched rows from only right:

In [96]:
# right join, merges all matched data, and only fills unmatched
# item from the right with NaN filled for the unmatched items
# in the result
# rows with labels 0 and 1 match on key1 and key2
# the row with label 2 is from right
left.merge(right, how='right')

  key1 key2  lval1  rval1
0    a    x    0.0      6
1    b    a    NaN      7
2    c    z    2.0      8

The pandas library also provides a <span style="color:red">.join()</span> method that can be used to perform a join using the index labels of the two DataFrame objects (instead of values in columns).<mark> Note that if the columns in the two DataFrame objects do not have unique column names, you must specify suffixes using the lsuffix and rsuffix parameters (automatic suffixing is not performed). The following code demonstrates both the join and specification of suffixes:

In [97]:
# join left with right (default method is outer)
# and since these DataFrame objects have duplicate column names
# we just specify lsuffix and rsuffix
left.join(right, lsuffix='_left', rsuffix='_right')

  key1_left key2_left  lval1 key1_right key2_right  rval1
0         a         x      0        NaN        NaN    NaN
1         b         y      1          a          x    6.0
2         c         z      2          b          a    7.0

The default type of join performed is an outer join. Note that this differs from the default of the .merge() method, which defaults to inner. To change to an inner join, specify how='inner', as is demonstrated in the following example:

In [98]:
left.join(right, lsuffix='_left', rsuffix='_right', how='inner')  # join left with right with an inner join

  key1_left key2_left  lval1 key1_right key2_right  rval1
1         b         y      1          a          x      6
2         c         z      2          b          a      7

 Notice that this is roughly equivalent to the earlier result from In[92] except with the result having columns with slightly different names.
 
It is also possible to perform right and left joins, but they lead to results similar to previous examples, so they will be omitted for brevity

<div class="alert alert-success" >
    
## <div class= "alert alert-info">Pivoting
    
Data is often stored in a stacked format, which is also referred to as record format; this is common in databases, .csv files, and Excel spreadsheets. In a stacked format, the data is often not normalized and has repeated values in many columns, or values that should logically exists in other tables (violating another concept of tidy data). Take the following data, which represents a stream of data from an accelerometer on a mobile device (provided with the data from the sample code):

In [99]:
path = 'Datasets/accel.csv'
sensor_readings = pd.read_csv(path) # read in accellerometer data
sensor_readings

    interval axis  reading
0          0    x      0.0
1          0    y      0.5
2          0    z      1.0
3          1    x      0.1
4          1    y      0.4
..       ...  ...      ...
7          2    y      0.3
8          2    z      0.8
9          3    x      0.3
10         3    y      0.2
11         3    z      0.7

[12 rows x 3 columns]

### <div class= "alert alert-danger">

In [1]:
#   15:54        https://www.youtube.com/watch?v=V0h4VqhRNy4